# Основы работы с количественными данными

*Алла Тамбовцева*

## Практикум 5. Доверительные интервалы и проверка гипотез


### Подготовка к работе

Загрузим необходимые библиотеки, модули и функции:

* библиотеку `numpy` для работы с массивами;
* библиотеку `pandas` для загрузки данных и работы с таблицами;
* модуль `stats` из библиотеки `scipy` (от *Scientific Python*) для построения доверительных интервалов и проверки статистических гипотез;
* функция `proportions_ztest` из библиотеки `statsmodels` для проверки гипотезы о доле.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

from statsmodels.stats.proportion import proportions_ztest

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Часть 0. Проверка статистических гипотез на примерах простых массивов

#### Проверка гипотезы о среднем

Допустим, на основе опроса 40 случайно выбранных студентов Вышки у нас есть информация о том, сколько чашек кофе студенты суммарно выпивают во время сессии. Результаты этого опроса – число чашек – сохранены в массиве `coffee`:

In [2]:
# числа большие, так как пример утрированный
# расчет такой: сессия 7 дней, по 3-4 чашки в день
# кофе необязательно крепкий, не переживайте за здоровье студентов

coffee = np.array([2, 3, 2, 3, 30, 23, 16, 13, 13, 10, 25, 18, 20, 14, 20, 
          28, 11, 29, 3, 31, 18, 17, 11, 7, 34, 14, 1, 20, 1, 17, 
          11, 25, 34, 7, 12, 34, 34, 18, 8, 22])

Проверим гипотезу о том, что среднее число чашек кофе, выпиваемых всеми студентами Вышки, равно 18 против двусторонней альтернативы (среднее не равно 18). Итак:

$$
H_0: \mu = 18
$$
$$
H_1: \mu \ne 18
$$

Для проверки такой гипотезы нам нужен критерий Стьюдента для одной выборки (он же t-критерий для одной выборки, потому что распределение Стьюдента и t-распределение – это одно и то же). Для этого нам понадобится функция с вполне логичным названием `ttest_1samp()` из модуля `stats`, то есть *t-test for one sample*. На первом месте внутри этой функции указываем массив, на втором – значение из нулевой гипотезы (так как в гипотезе у нас среднее генеральной совокупности, здесь это называется `popmean`, то есть *population mean*).

In [3]:
# проинтерпретируем
# и подумаем, что бы изменилось, если бы альтернатива была односторонней

st.ttest_1samp(coffee, popmean = 18)

Ttest_1sampResult(statistic=-0.9455346732709895, pvalue=0.3502076174156046)

На первом месте в результатах выводится наблюдаемое значение статистики критерия $t_{набл}$, то есть разность между выборочным средним, посчитанным по массиву, и средним из гипотезы 18, которую мы делим на стандартную ошибку среднего. Здесь $t_{набл} = -0.945$.

В целом, по этому значению мы можем понять, отвергнуть нулевую гипотезу или нет, зная, какие значения для $t$ считаются типичными, а какие – нетипичными. Так, если мы проверяем гипотезу на 5%-ном уровне значимости и объём выборки достаточно большой (30 наблюдений и больше):

* если значение $t_{набл}$ лежит в интервале от $-2$ до $2$, оно считается типичным при условии, если нулевая гипотеза верна, а значит, отвергать гипотезу на основе имеющихся данных не стоит;

* если значение $t_{набл}$ выходит за границы этого интервала, оно считается нетипичным для случая, если нулевая гипотеза верна, а значит, данные дают основания эту гипотезу отвергнуть.

**Интерпретация 1. В нашем случае значение $t_{набл} = -0.945$ лежит в интервале от $-2$ до $2$, значит, нулевая гипотеза не отвергается, а значит, действительно, среднее число чашек кофе, выпиваемых студентами во время сессии, равно 18.** 

Однако на практике мало кто обращает внимание на значение статистики, потому что есть более универсальный показатель – p-value, мера «жизнеспособности» нулевой гипотезы с точки зрения имеющихся данных. Выдача с результатами теста в Python тоже включает это значение, здесь p-value = 0.35. Если мы проверяем нулевую гипотезу на 5%-ном уровне значимости, мы должны сравнить p-value с 0.05:

* если p-value больше 0.05, то данные свидетельствуют в пользу нулевой гипотезы (ее жизнеспособность высока, выходит за пределы допустимой погрешности в 5%), значит, гипотезу отвергать не стоит;

* если p-value меньше 0.05, то данные не поддерживают нулевую гипотезу, значит, ее следует отвергнуть. 

**Интерпретация 2. В нашем случае p-value = 0.35, поэтому нулевую гипотезу мы не отвергаем и вновь соглашаемся с тем, что среднее число чашек кофе, выпиваемых студентами во время сессии, равно 18.**  

**Вопрос:** а каким было бы p-value, если бы мы тестировали гипотезу против *односторонней альтернативы*? То есть:

$$
H_0: \mu = 18
$$
$$
H_1: \mu < 18
$$

Мы обсуждали, что если посмотреть на p-value более формально, это вероятность получить значение $t$ еще более нетипичное, чем то, которое мы получили на наших данных, а оно у нас $-0.945$. Когда альтернативная гипотеза была двусторонней, p-value считалось так: 

$$
\text{p-value} = P(t < -0.945) + P(t > 0.945) = 2 \times P(t < -0.945) \approx 0.35
$$

Пояснения: раз двусторонняя, то отклониться мы можем и в большую, и в меньшую сторону, плюс, так как t-распределение симметричное, вероятности попасть в левый и правый «хвост» распределения одинаковые, можем взять какую-то одну и домножить на 2 (вспомните картинки с закрашенными площадями). 

Если теперь гипотеза у нас односторонняя, то отклонение допустимо только в одну сторону, в меньшую (поскольку в альтернативе знак `<`, она левосторонняя):

$$
\text{p-value} = P(t < -0.945) = 0.35 / 2 = 0.175
$$

В нашем случае при смене типа альтернативной гипотезы вывод не поменяется, p-value все равно высокое, больше 0.05. Однако это не всегда так, поэтому с типом альтернативы всегда нужно определяться в начале исследования.

**Резюмируем эти выкладки:**

* t-тесты в Python всегда запускаются с учетом двусторонних альтернатив;
* если нам нужна односторонняя альтернатива, p-value из выдачи делим на два.

#### Проверка гипотезы о равенстве двух средних

Допустим, сейчас мы хотим выяснить, совпадает ли среднее число чашек кофе, выпиваемых студентками и студентами Вышки. Для этого мы можем проверить гипотезу о равенстве средних (например, против двусторонней альтернативы):

$$
H_0: \mu_{female} = \mu_{male}
$$

$$
H_1: \mu_{female} \ne\mu_{male}
$$

Снова есть два массива, один с результатами опроса девушек, другой – с результатами опроса юношей:

In [4]:
coffee_female = np.array([30, 23, 25, 20, 20, 28, 29, 31, 34, 20, 25, 34, 34, 34, 22])
coffee_male = np.array([ 2,  3,  2,  3, 16, 13, 13, 10, 18, 20, 14, 20, 11, 3, 
                        18, 17, 11, 7, 14,  1, 20])

Для проверки такой гипотезы нам нужен критерий Стьюдента для двух независимых выборок (он же t-критерий для двух выборок). Для этого нам понадобится функция `ttest_ind()` из модуля `stats`, то есть *t-test for independent samples*. Почему выборки здесь независимые? Потому что это две группы разных людей. Если бы выборки были связанными (не независимыми, их еще называют парными), в них были бы одни и те же объекты, но в разные моменты времени. Примеры связанных выборок: одни и те же регионы до и после реформы, одни и те же люди в начале терапии и после, компании до изменения правил внутреннего распорядка и после).

Так как критерий по определению проверяет гипотезу о равенстве средних на основе двух выборок, кроме двух выборок нам ничего не понадобится подавать на вход этой функции, поэтому в скобках мы просто указываем два массива.

In [5]:
# проинтерпретируем

st.ttest_ind(coffee_female, coffee_male)

Ttest_indResult(statistic=7.613042281116699, pvalue=7.594644170555759e-09)

Вновь видим наблюдаемое значение статистики и p-value. Только в этот раз p-value записано странно. Это так называемый компьютерный формат числа. Чтобы не писать кучу знаков после точки, Python сокращает запись, и  здесь `e-09` означает $10^{-9}$. То есть p-value в нашем случае равно примерно $7.59 \times 10^{-9}$, что очень близко к 0. 

**Итого:** раз p-value почти 0, оно точно меньше 0.05, поэтому, если мы вновь проверяли нулевую гипотезу на 5%-ном уровне значимости (хотя тут на 10% и на 1% будет такой же вывод), эту гипотезу необходимо отвергнуть. Следовательно, на основе имеющих данных нельзя заключить, что девушки и юноши, в среднем, выпивают одинаковое число чашек кофе во время сессии. 

#### Проверка гипотезы о доле

Допустим, сейчас мы хотим выяснить, равна ли доля любителей рано вставать по утрам 0.25. Нулевая гипотеза:

$$
H_0: p = 0.25
$$

С альтернативной гипотезой мы пока не определились. Давайте выберем её направление, исходя из данных, а данные у нас есть – в виде бинарного массива `zhavoronki`, где 1 соответствуют любителям вставать рано по утрам:

In [6]:
zhavoronki = np.array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

Вычислим долю единиц:

In [7]:
# вспоминаем, что в таком бинарном случае
# доля единиц = среднее по массиву

p = zhavoronki.mean()
p

0.3076923076923077

Какую альтернативу выбрать, правостороннюю или левостороннюю? Раз, согласно гипотезе, ожидаемая доля равна 0.25, а на реальных данных мы получили долю побольше, равную 0.31, имеет смысл выбрать правосторонюю альтернативу. Итак: 

$$
H_0: p = 0.25
$$

$$
H_1: p > 0.25
$$

Чтобы проверить гипотезу о равенстве доли числу, нам понадобится функция `proportions_ztest()`, которую мы импортировали отдельно в самом начале (тесты для работы с долями называются z-тестами, поскольку значения статистик, которые используются для вычисления гипотез, принадлежат стандартному нормальному распределению, а оно еще называется z-распределением, так исторически сложилось).

Внутри модуля `stats` такой функции нет, поэтому мы «позвали» ее из другой библиотеки. Она принимает на вход следующие аргументы:

* `count`: число успехов (число единиц в нашем случае);
* `nobs`: общее число наблюдений;
* `value`: значение доли из нулевой гипотезы;
* `alternative`: тип альтернативы, по умолчанию двусторонняя (`"two-sided"`), можно изменить на левостороннюю (`"smaller"`) или правосторонюю (`"larger"`).

Давайте поработаем с массивом `zhavoronki` и подставим необходимые значения в код ниже.

In [8]:
# count – число единиц – считаем сумму по массиву zhavoronki
# nobs – число наблюдений – число элементов в массиве size
# value – p из гипотезы
# alternative – договорились на Н со знаком больше, отсюда larger

proportions_ztest(count = zhavoronki.sum(), 
                  nobs = zhavoronki.size, 
                  value = 0.25,
                  alternative = "larger")

(0.6373774391990983, 0.26193951009734207)

Так же, как и предыдущие тесты, эта функция возвращает наблюдаемое значение статистики и p-value, однако не подписывает, что есть что. Но, по-прежнему, первое значение – это значение статистики (здесь $z_{набл}$), а второе – это p-value. 

Здесь p-value равно 0.26, сильно больше 0.05, поэтому нулевую гипотезу мы не отвергаем. Значит, долю любителей рано вставать по утрам, действительно, можно считать равной 0.25 (на наших данных она чуть больше, 0.31, но разница между 0.25 и 0.31 считается незначительной с учётом потенциального разброса данных).

### Часть 1: изучаем героев и актёров, проверяем гипотезу о доле и среднем

Вернёмся к результатам сказочного [опроса](https://forms.gle/WoCyKKySQJrDAoUWA) и загрузим данные из отдельных CSV-файлов, размещённых на Github. Отдельные CSV с компьютера мы уже загружали, листы из файлов Excel тоже, теперь это снова отдельные CSV-файлы, но теперь уже мы пишем не путь к файлам, а ссылки на них. Сделано это для удобства, чтобы нам не пришлось отвлекаться на работу с разными файлами.

Получим несколько датафреймов – таблиц с данными:

* `start`: выбор героев по текстовому описанию;
* `end`: итоговый выбор героев;
* `likes`: лайки-дизлайки, поставленные по кадрам;
* `actors`: общая информация по актёрам.

In [9]:
# везде одна и та же функция read_csv()
# но загружаем разные файлы

start = pd.read_csv("https://raw.githubusercontent.com/allatambov/QuantData23/main/poll_description.csv")
end = pd.read_csv("https://raw.githubusercontent.com/allatambov/QuantData23/main/poll_final.csv")
likes = pd.read_csv("https://raw.githubusercontent.com/allatambov/QuantData23/main/poll_likes.csv")
actors = pd.read_csv("https://raw.githubusercontent.com/allatambov/QuantData23/main/actors.csv")

Датафреймы `start`, `end`, `likes` устроены одинаково (формат один, героя выбрали или нет, но выбор разный, по описанию, по кадрам и итоговый):

In [10]:
# первые 5 строк start
start.head()

,Unnamed: 0,id,группа,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
0,0,35,221_223,политология,жен,0,1,1,0,0,0,0,0,0,0
1,1,36,221_223,политология,жен,0,0,0,0,0,1,0,1,0,0
2,2,37,221_223,политология,жен,0,0,0,0,0,0,0,1,0,0
3,3,38,221_223,политология,жен,0,0,0,1,1,0,0,0,0,0
4,4,39,221_223,политология,муж,0,0,0,0,0,1,1,0,0,0


In [11]:
# первые 5 строк end
end.head()

,Unnamed: 0,id,группа,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
0,0,35,221_223,политология,жен,0,0,0,0,0,1,1,0,0,0
1,1,36,221_223,политология,жен,0,0,0,0,0,0,1,1,0,0
2,2,37,221_223,политология,жен,0,0,0,0,0,0,1,1,0,0
3,3,38,221_223,политология,жен,0,0,0,0,0,0,0,0,1,1
4,4,39,221_223,политология,муж,0,0,0,0,0,1,1,0,0,0


In [12]:
# первые 5 строк likes
likes.head()

,Unnamed: 0,id,группа,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
0,0,35,221_223,политология,жен,1,0,1,1,0,0,1,1,1,1
1,1,36,221_223,политология,жен,0,0,1,1,0,1,1,1,1,1
2,2,37,221_223,политология,жен,0,0,1,1,0,0,1,1,1,0
3,3,38,221_223,политология,жен,0,1,0,1,0,0,1,1,1,1
4,4,39,221_223,политология,муж,0,0,0,1,0,1,1,1,0,0


Во всех датафреймах:

* `id`: это номер респондента;
* `группа`: это учебная группа;
* `профиль`: профиль (специализация) респондента;
* `пол`: пол респондента;
* `Теодор`–`Марселла`: бинарные индикаторы (1 – студент выбрал героя, 0 – не выбрал).

Вспомним, что мы делали в прошлый раз и посчитаем суммы по столбцам из 0 и 1, чтобы выяснить, какие герои понравились больше, а какие – меньше. Для этого применим ко всему датафрейму метод `.sum()`, и он посчитает сумму по всем столбцам сразу (`numeric_only = True` – чтобы были выбраны только числовые столбцы): 

In [13]:
# результаты вдобавок отсортируем – после sum допишем .sort_values()
# ascending = False – сортировка по убыванию

print("Выбор на основе текстового описания:")
start.sum(numeric_only = True).sort_values(ascending = False)

Выбор на основе текстового описания:


Unnamed: 0    5886
Патрик          31
Давиль          29
Жак             29
Марта           29
Альбина         22
Оттилия         20
Марселла        15
Флора           14
Пенапью         13
Теодор           5
dtype: int64

На строку `Unnamed: 0` можно не обращать внимание, при выгрузке в CSV-файл автоматически добавился номер строки, это он и есть, а сумма номеров строк нас вряд ли заинтересует. 

Итак, топ-3 по текстовому описанию героев: Патрик > Давиль ~ Жак ~ Марта. Проделаем то же самое для итогового выбора:

In [14]:
print("Итоговый выбор:")
end.sum(numeric_only = True).sort_values(ascending = False)

Итоговый выбор:


Unnamed: 0    5886
Оттилия         45
Патрик          39
Марта           32
Жак             31
Давиль          25
Марселла        12
Альбина          8
Теодор           7
Пенапью          4
Флора            3
dtype: int64

Вот тут топ-3 немного другой: Оттилия > Патрик > Марта. Проверим лайки:

In [15]:
print("Число лайков, полученных актёрами в образах героев:")
likes.sum(numeric_only = True).sort_values(ascending = False)

Число лайков, полученных актёрами в образах героев:


Unnamed: 0    5886
Марта           97
Жак             95
Патрик          86
Оттилия         83
Альбина         69
Теодор          66
Давиль          66
Марселла        55
Флора           53
Пенапью         39
dtype: int64

Топ-3: Марта > Жак > Патрик.

А что, если мы хотим какой-то сводный показатель для героев, вроде рейтинга? Порядковый рейтинг нам построить не из чего, но никто не мешает просуммировать бинарные столбцы и получить сводный индекс «популярности»:

In [16]:
# тут без сортировки, чтобы все герои везде шли в одинаковом порядке

start_tab = start.sum(numeric_only = True)
end_tab = end.sum(numeric_only = True)
likes_tab = likes.sum(numeric_only = True)

In [17]:
# столбцы для героев в одинаковом порядке 
# можно просто сложить!

fin_tab = start_tab + end_tab + likes_tab
fin_tab

Unnamed: 0    17658
Теодор           78
Флора            70
Альбина          99
Патрик          156
Пенапью          56
Давиль          120
Оттилия         148
Жак             155
Марта           158
Марселла         82
dtype: int64

*Примечание по коду:* столбцы из таблиц можно смело складывать друг с другом, Python поймет, что нужно первый элемент первого столбца сложить с первым элементом первого столбца, второй – со вторым, и так далее. С массивами можно поступать точно так же, если они одинаковой длины. 

In [18]:
# а теперь уже отсортируем, чтобы посмотреть, что получилось

fin_tab.sort_values(ascending = False)

Unnamed: 0    17658
Марта           158
Патрик          156
Жак             155
Оттилия         148
Давиль          120
Альбина          99
Марселла         82
Теодор           78
Флора            70
Пенапью          56
dtype: int64

В общем «зачёте» отрицательные герои вытеснились из топа-3, работаем дальше.

Давайте перейдем к долям и вспомним про доверительные интервалы и проверку гипотез! Но сначала проделаем следующее: имитируем реальное исследование! Предположим, что на основе предыдущих опросов (а они действительно были) мы хотим сформулировать какую-то гипотезу, а затем проверить её на данных, полученных в новом опросе (опрос вашей группы). 

Для этого разделим нашу таблицу с итоговыми результатами `end` на две части – результаты прошлых опросов и результаты нового, только по нашей группе (наша группа – *гк*, я так сократила *госкоммуникации* как кальку с вашего адреса почты). В таблицу `old` сохраним строки с результатами предыдущих опросов, то есть до нашей группы:

In [19]:
old = end[end["группа"] != "гк"]
old

,Unnamed: 0,id,группа,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
0,0,35,221_223,политология,жен,0,0,0,0,0,1,1,0,0,0
1,1,36,221_223,политология,жен,0,0,0,0,0,0,1,1,0,0
2,2,37,221_223,политология,жен,0,0,0,0,0,0,1,1,0,0
3,3,38,221_223,политология,жен,0,0,0,0,0,0,0,0,1,1
4,4,39,221_223,политология,муж,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,409,эк,экономика,жен,0,0,0,1,0,0,0,1,0,0
95,95,410,эк,экономика,жен,0,1,0,0,0,0,1,0,0,0
96,96,411,эк,экономика,муж,0,0,0,1,0,0,0,0,1,0
97,97,412,эк,экономика,жен,0,0,0,0,0,1,1,0,0,0


Пояснения к коду:
    
* отбираем строки, где в столбце `группа` стоит любое значение, кроме *гк* (вспоминаем, не равно – это`!=`);
* код `end["группа"] != "гк"` возвращает массив из `True` и `False` (выполняется ли условие или нет, и так для каждой строки таблицы);
* поместив этот код внутрь квадратных скобках, мы выполнили фильтрацию – выбрали те строки, на которых было возвращено `True`.

Аналогичным образом выберем строки, соответствующие вашей группе, то есть те, где в столбце `группа` указано *гк*:

In [20]:
new = end[end["группа"] == "гк"]
new

,Unnamed: 0,id,группа,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
99,99,r1,гк,NaN,жен,0,0,0,0,0,0,0,1,1,0
100,100,r2,гк,NaN,жен,0,0,0,1,0,0,0,0,0,1
101,101,r3,гк,NaN,жен,0,0,0,1,1,0,0,0,0,0
102,102,r4,гк,NaN,муж,0,0,0,1,0,1,0,0,0,0
103,103,r5,гк,NaN,жен,0,0,0,0,0,0,1,1,0,0
104,104,r6,гк,NaN,муж,0,0,0,1,0,0,1,0,0,0
105,105,r7,гк,NaN,жен,0,0,0,0,0,1,1,0,0,0
106,106,r8,гк,NaN,жен,1,0,0,0,0,1,0,0,0,0
107,107,r9,гк,NaN,муж,0,0,0,0,0,0,1,0,1,0
108,108,r10,гк,NaN,жен,0,0,0,0,0,1,1,0,0,0


Теперь давайте выберем какого-нибудь героя и проверим гипотезу о доле людей, проголосовавших за него. Откуда нам взять значение доли для гипотезы? Обычно это значение берут либо на основании предыдущих исследований, либо на основе экспертных оценок. Экспертные оценки мы вряд ли найдем, а вот результаты предыдущих опросов у нас есть – это датафрейм `old`.

На паре было принято решение – изучить долю студентов, проголосовавших за Патрика. Для этого выберем соответствующий ему столбец из `old` и посчитаем по нему среднее:

In [21]:
old["Патрик"].mean()

0.35353535353535354

Если мы предположим, что студенты в нашей группе ничем принципиально не отличаются от студентов других специальностей, опрошенных ранее (а это так, там студенты магистратуры и бакалавриата Вышки, преимущественно социально-экономических и гуманитарных специальностей), мы можем сформулировать гипотезу о том, что доля студентов, проголосовавших за Патрика, не отличается от 0.35. Итак:

$$
H_0: p = 0.35
$$

$$
H_1: p \ne 0.35
$$

Проверяем гипотезу, уже на данных нашей группы, это датафрейм `new`. Так как гипотеза о равенстве доли числу, нам снова нужна функция `proportions_ztest()`, альтернатива у нас двусторонняя:

In [22]:
# count – число 1, то есть сумма по столбцу в new
# nobs – число наблюдений, это size
# value – из гипотезы выше
# alternative – two-sided, так как двустороняя

proportions_ztest(count = new["Патрик"].sum(), 
                  nobs = new["Патрик"].size, 
                  value = 0.35,
                  alternative = "two-sided")

(0.3227486121839517, 0.7468856333903637)

Первое значение – наблюдаемое значение статистики – попадает в интервал от $-$2 до 2, это сигнал к тому, что с точки зрения полученных данных наша нулевая гипотеза имеет право на жизнь. Но нам интереснее второе значение, это p-value, p-value здесь сильно больше 0.05, поэтому на 5%-ном уровне значимости нулевая гипотеза не отвергается. Долю студентов, которым понравился Патрик, можно считать равной 0.35. А это значит, что наша группа по предпочтениям относительно этого героя не отличается от всех остальных, опрошенных ранее :)

**Дополнительно для желающих:** функция выше не зря называется `proportions_`, а не `proportion`, с ее помощью можно также сравнивать доли между собой (то есть проверять гипотез о равенстве двух долей). В таком случае в `count` нужно вписывать два значения числа успехов в виде списка, например, число «единиц» в первой и второй группе, а в `nobs` – два значения объёма выборок в виде списка. 

Для примера сравним доли проголосовавших за Патрика и Жака в предыдущих опросах (там просто данных больше, 10 опрошенных в `new` делить на группы несерьезно):

In [23]:
# разница в долях не 0 (если была бы 0, значение статистики 
# на первом месте было бы 0,
# но при этом гипотезу о равенстве долей все равно не отвергаем
# друзья детства (по сюжету) на одном уровне одобрения :)

proportions_ztest(count = [old["Патрик"].sum(), old["Жак"].sum()],
                  nobs = [old["Патрик"].size, old["Жак"].size], 
                  alternative = "two-sided")

(0.9116779674961496, 0.3619382655414811)

### Часть 2: изучаем студентов, агрегируем данные и проверяем гипотезу о равенстве средних

Теперь поизучаем студентов и их выбор в зависимости от профиля и пола. Более детально изучать зависимости мы будем в следующий раз (и наконец, завершим историю с этим опросом, интересную с психологической и антропологической точки зрения), а пока просто построим сводный индекс одобрения героев и сравним его значения в разных группах.

Итак, на входе по каждому герою у нас есть три набора данных из 0 и 1: одобрение/неодобрение по текстовому описанию, одобрение/неодобрение по кадрам и одобрение/неодобрение при итоговом выборе. Никто не мешает объединить эти наборы простым суммированием и получить единый индекс одобрения героев. Если герой совсем не понравился респонденту, по всем трем измерениям у него будут нули, если понравился во всех аспектах, по всем трем измерениям у него будут единицы. В итоге при суммировании измерений мы получим вполне себе количественный индекс, принимающий целые значения от 0 до 3. 

Проделаем эти манипуляции для всех героев (не закапываемся в программирование и циклы, поэтому честно пишем 10 строчек, по одному для каждого героя):

In [24]:
king = start["Теодор"] + end["Теодор"] + likes["Теодор"]
queen = start["Флора"] + end["Флора"] + likes["Флора"]
princess = start["Альбина"] + end["Альбина"] + likes["Альбина"]
poet = start["Патрик"] + end["Патрик"] + likes["Патрик"]
prince = start["Пенапью"] + end["Пенапью"] + likes["Пенапью"]
kanzler = start["Давиль"] + end["Давиль"] + likes["Давиль"]
kanzler_wife = start["Оттилия"] + end["Оттилия"] + likes["Оттилия"]
actor = start["Жак"] + end["Жак"] + likes["Жак"]
actress = start["Марта"] + end["Марта"] + likes["Марта"]
maid = start["Марселла"] + end["Марселла"] + likes["Марселла"]

Посколько здесь нас интересуют не столько сами герои, сколько то, какие студенты их выбирали чаще всего, заберем из датафрейма `start` столбцы с характеристиками самих респондентов – профиль и пол:

In [25]:
# могли бы забрать из end или likes
# студенты везде одни и те же

spec = start["профиль"]
sex = start["пол"]

Объединим всё в новую таблицу, которая позволит отвечать на вопросы о различиях в предпочтениях студентов:

In [26]:
# код ниже тяжеловат для восприятия, если страшно, пропускайте,
# тут важно то, что мы получим на выходе

# DataFrame – объединяет столбцы/массивы в одну таблицу
# эти столбцы/массивы перечисляем в квадратных скобках (это список)
# .T в конце 

data = pd.DataFrame([spec, sex, king, queen, princess, poet, prince, 
              kanzler, kanzler_wife, actor, actress, maid]).T
data

,профиль,пол,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
0,политология,жен,1,1,2,1,0,1,2,1,1,1
1,политология,жен,0,0,1,1,0,2,2,3,1,1
2,политология,жен,0,0,1,1,0,0,2,3,1,0
3,политология,жен,0,1,0,2,1,0,1,1,2,2
4,политология,муж,0,0,0,1,0,3,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,NaN,муж,0,0,1,3,0,1,3,0,0,0
105,NaN,жен,0,0,1,0,0,3,3,1,1,1
106,NaN,жен,2,2,2,1,1,2,1,1,1,2
107,NaN,муж,0,1,1,1,0,1,2,1,2,2


На выходе мы получаем таблицу, где одна строка соответствует одному студенту, а в столбцах идут значения индекса одобрения героев (значения от 0 до 3).

Вспомним группировку и агрегирование и посчитаем среднее значение индекса одобрения героев по каждому профилю:

In [27]:
# в groupby – название столбца, по которому группируем
# в agg – функция для агрегирования, здесь среднее

data.groupby("профиль").agg("mean")

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
профиль,,,,,,,,,,
бизнес-информатика,0.774194,0.774194,1.000000,1.322581,0.516129,0.903226,1.580645,1.451613,1.645161,0.612903
политология,0.733333,0.466667,0.866667,1.511111,0.400000,1.266667,1.222222,1.444444,1.244444,0.577778
психология,0.600000,1.000000,1.000000,1.500000,0.400000,0.800000,1.000000,1.700000,2.000000,1.100000
экономика,0.692308,0.692308,0.769231,1.307692,0.769231,1.076923,1.307692,1.307692,1.461538,1.076923


А теперь с делением по полу:

In [28]:
data.groupby("пол").agg("mean")

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


,Теодор,Флора,Альбина,Патрик,Пенапью,Давиль,Оттилия,Жак,Марта,Марселла
пол,,,,,,,,,,
жен,0.688525,0.819672,0.934426,1.344262,0.52459,0.901639,1.508197,1.393443,1.491803,0.868852
муж,0.750000,0.425000,0.825000,1.575000,0.55000,1.400000,1.100000,1.400000,1.400000,0.625000


Можете пока порефлексировать над результатами в общем виде (или попроверять гипотезы, например, о сравнении средних в двух группах), к ним мы вернемся позже в рамках проверки гипотезы о независимости признаков.